<a href="https://colab.research.google.com/github/96jonesa/CSE-517-Project/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
!pip3 install --quiet "tensorflow-hub>=0.7.0"
!pip3 install --quiet seaborn
!pip3 install --quiet pandas-market-calendars

In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from absl import logging
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import re
import seaborn as sns
import json
import itertools
import pandas as pd
import torch
import pandas_market_calendars as mcal
import datetime
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

#Layers

In [34]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, batch_first=False):
        super(GRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_first = batch_first

        self.gru = nn.GRU(input_size, hidden_size, batch_first=self.batch_first)

    def forward(self, input):
        output, hn = self.gru(input)
        return output, hn

In [4]:
# attention weights are softmax(u^T tanh(W input + b)) where W is learned parameter matrix, u is a learned parameter vector, and b is a learned offset

class LinearAttention(nn.Module):
    def __init__(self, input_size, intermediate_size, weights_size):
        super(LinearAttention, self).__init__()
        self.input_size = input_size
        self.intermediate_size = intermediate_size
        self.weights_size = weights_size

        self.linear_1 = nn.Linear(self.input_size, self.intermediate_size, bias=True)
        self.linear_2 = nn.Linear(self.intermediate_size, self.weights_size, bias=False)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=2)

    def forward(self, input, mask=None):
        intermediate = self.tanh(self.linear_1(input))
        pre_attention = self.linear_2(intermediate)
        if mask is not None:
            zero_vec = -9e15*torch.ones_like(pre_attention)
            pre_attention = torch.where(mask > 0, pre_attention, zero_vec)
        attention_weights = self.softmax(pre_attention)
        attention_weights = attention_weights.permute(0, 2, 1)
        output_features = torch.bmm(attention_weights, input)

        return output_features

In [35]:
# output is ReLU(left^T W right + b) where W is a learned paramater matrix
# and b is a learned bias

class Blend(nn.Module):
    def __init__(self, left_size, right_size, output_size):
        super(Blend, self).__init__()
        self.left_size = left_size
        self.right_size = right_size
        self.output_size = output_size

        self.bilinear = nn.Bilinear(self.left_size, self.right_size, output_size, bias=True)
        self.relu = nn.ReLU()
    
    def forward(self, left, right):
        output = self.relu(self.bilinear(left, right))

        return output

In [36]:
# https://github.com/Diego999/pyGAT/blob/master/layers.py

"""class SGAT(nn.Module):
    def __init__(self, input_size, output_size, leakyrelu_slope=0.01):
        super(SGAT, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.leakyrelu_slope = leakyrelu_slope
        
        self.W = nn.Parameter(torch.empty(size=(input_size, output_size)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*output_size, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        self.leakyrelu = nn.LeakyReLU(self.leakyrelu_slope)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        h_prime = torch.matmul(attention, Wh)

        return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        N = Wh.size()[0] # number of nodes
        
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)
        Wh_repeated_alternating = Wh.repeat(N, 1)

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)

        return all_combinations_matrix.view(N, N, 2 * self.output_size)"""

'class SGAT(nn.Module):\n    def __init__(self, input_size, output_size, leakyrelu_slope=0.01):\n        super(SGAT, self).__init__()\n        self.input_size = input_size\n        self.output_size = output_size\n        self.leakyrelu_slope = leakyrelu_slope\n        \n        self.W = nn.Parameter(torch.empty(size=(input_size, output_size)))\n        nn.init.xavier_uniform_(self.W.data, gain=1.414)\n        self.a = nn.Parameter(torch.empty(size=(2*output_size, 1)))\n        nn.init.xavier_uniform_(self.a.data, gain=1.414)\n        self.leakyrelu = nn.LeakyReLU(self.leakyrelu_slope)\n\n    def forward(self, h, adj):\n        Wh = torch.mm(h, self.W)\n        a_input = self._prepare_attentional_mechanism_input(Wh)\n        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))\n\n        zero_vec = -9e15*torch.ones_like(e)\n        attention = torch.where(adj > 0, e, zero_vec)\n        attention = F.softmax(attention, dim=1)\n        h_prime = torch.matmul(attention, Wh)\n\n    

In [55]:
class MANSF(nn.Module):
    def __init__(self, T, gru_hidden_size, attn_inter_size, use_embed_size,
                 blend_size, gat_1_inter_size, gat_2_inter_size, leakyrelu_slope, elu_alpha, U):
        super(MANSF, self).__init__()
        self.T = T
        self.gru_hidden_size = gru_hidden_size
        self.attn_inter_size = attn_inter_size
        self.use_embed_size = use_embed_size
        self.blend_size = blend_size
        #self.gat_1_inter_size = gat_1_inter_size
        #self.gat_2_inter_size = gat_2_inter_size
        self.leakyrelu_slope = leakyrelu_slope
        self.elu_alpha = elu_alpha
        self.U = U

        self.gru_p = GRU(3, gru_hidden_size, batch_first=True)
        self.gru_m = GRU(use_embed_size, gru_hidden_size, batch_first=True)
        self.gru_s = GRU(gru_hidden_size, gru_hidden_size, batch_first=True)
        #self.attn_p = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        #self.attn_m = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        #self.attn_s = LinearAttention(gru_hidden_size, attn_inter_size, 1)
        self.blend = Blend(gru_hidden_size, gru_hidden_size, blend_size)
        #self.mgat_1 = nn.ModuleList([SGAT(blend_size, gat_1_inter_size, leakyrelu_slope=leakyrelu_slope) for u in range(U)])
        #self.mgat_2 = nn.ModuleList([SGAT(U * gat_1_inter_size, gat_2_inter_size, leakyrelu_slope=leakyrelu_slope) for u in range(U)])
        self.sigmoid = nn.Sigmoid()
        self.elu = nn.ELU(elu_alpha)
        #self.final_linear = nn.Linear(U * gat_2_inter_size, 1, bias=True)
        self.final_linear = nn.Linear(blend_size, 1, bias=True)

    # p is price data tensor of shape (num_stocks, T, 3), for the day under consideration
    # m is smi data list of tensors of shape (num_stocks, K, use_embed_size) of length T,
    #       where K is the number of tweets for the given stock on the day under consideration
    # neighorhoods is a list of adjacency lists, where each stock is indexed with the same
    #       indices they have in p and m
    def forward(self, p, m, m_mask, neighborhoods):
        ## price encoding
        #h_p, _ = self.gru_p(p)
        _, h_p = self.gru_p(p)
        #q = self.attn_p(h_p)

        ## smi encoding (day level)
        r = torch.zeros(p.shape[0], 0, self.gru_hidden_size)
        r = r.to(device)
        for t in range(self.T):
            h_m, _ = self.gru_m(m[t])
            #r_t = self.attn_m(h_m, m_mask[t])
            r = torch.cat((r, h_m), 1)

        ## smi encoding (aggregate)
        #h_s, _ = self.gru_s(r)
        _, h_s = self.gru_s(r)
        #c = self.attn_s(h_s)

        ## blending
        #x = self.blend(q, c)
        x = self.blend(h_p, h_s)

        ## reshaping (eliminating superfluous dimension)
        #x = x.view(x.shape[0], x.shape[2])
        x = x.squeeze()

        ## first gat layer
        #  first head
        """sgat = self.mgat_1[0]
        z = sgat(x, neighborhoods)
        z = self.elu(z)

        #  remaining heads
        for u in range(1, self.U):
            sgat = self.mgat_1[u]
            z_u = sgat(x, neighborhoods)
            z_u = self.elu(z_u)
            
            z = torch.cat((z, z_u), 1)
        
        ## second gat layer
        #  first head
        sgat = self.mgat_2[0]
        new_z = sgat(z, neighborhoods)
        new_z = self.sigmoid(new_z)

        #  remaining heads
        for u in range(1, self.U):
            sgat = self.mgat_2[u]
            new_z_u = sgat(z, neighborhoods)
            new_z_u = self.sigmoid(new_z_u)
            
            new_z = torch.cat((new_z, new_z_u), 1)"""
        
        ## final layer
        #y = self.sigmoid(self.final_linear(new_z))
        y = self.sigmoid(self.final_linear(x))

        ## return result
        return y

#Data Processing

In [13]:
#!wget https://github.com/yumoxu/stocknet-dataset/archive/master.zip

In [14]:
#!unzip master.zip

In [38]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [39]:
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

In [17]:
stocknet_dataset_filepath = './stocknet-dataset-master'
train_start_date = '2014-01-01'
train_end_date = '2015-07-31'
val_start_date = '2015-08-01'
val_end_date = '2015-09-30'
test_start_date = '2015-10-01'
test_end_date = '2016-01-01'

In [10]:
def prep_dataset(dataset_filepath, start_date, end_date):
    cache = {}
    calendar = mcal.get_calendar('NYSE')
    def next_trading_day(start_day=None, SAFE_DELTA = 4):
        """Returns the next/previous trading date separated by a certain number of 
        trading days.
        """
        if start_day is None:
            start_day = datetime.datetime.utcnow().date()
        if start_day in cache:
            return cache[start_day]
        start = pd.to_datetime(start_day)
        end = start + np.timedelta64(SAFE_DELTA, 'D')
        business_days = calendar.valid_days(start_date=start, end_date=end)
        next_day = business_days[1].date()
        next_day = next_day.strftime("%Y-%m-%d")
        cache[start_day] = next_day
        return next_day
    
    raw_prices_filepath = stocknet_dataset_filepath + '/price/raw'
    preprocessed_tweets_filepath = stocknet_dataset_filepath + '/tweet/preprocessed'

    company_to_price_df = {}
    company_to_tweets = {}

    for filename in os.listdir(raw_prices_filepath):
        with open(raw_prices_filepath + '/' + filename) as file:
            company_name = filename.split('.')[0]
            
            # Not enough data for GMRE
            if company_name == 'GMRE':
                continue
            df = pd.read_csv(file)
            df.columns = ['date', 'open', 'high', 'low', 'close', 'adjust_close', 'volume']
            mask = (df['date'] >= start_date) & (df['date'] <= end_date)
            df = df.loc[mask]
            company_to_price_df[company_name] = df.dropna()

    for filename in tqdm(os.listdir(preprocessed_tweets_filepath)):
        company_name = filename.split('.')[0]
        dates_to_tweets = {}
        for tweet_filename in os.listdir(preprocessed_tweets_filepath + '/' + filename):
            if tweet_filename < start_date or tweet_filename > end_date:
                continue
            with open(preprocessed_tweets_filepath + '/' + filename + '/' + tweet_filename) as file:
                list_of_tweets = []
                for line in file:
                    tweet_json = json.loads(line)
                    list_of_tweets.append(tweet_json)
                date_idx = next_trading_day(tweet_filename)
                if date_idx not in dates_to_tweets:
                    dates_to_tweets[date_idx] = list_of_tweets
                else:
                    dates_to_tweets[date_idx] += list_of_tweets
        company_to_tweets[company_name] = dates_to_tweets
    
    # Reduce logging output.
    logging.set_verbosity(logging.ERROR)
    tf.get_logger().setLevel(logging.ERROR)
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    # Import the Universal Sentence Encoder's TF Hub module
    def embed_useT(module):
        with tf.Graph().as_default():
            sentences = tf.placeholder(tf.string)
            embed = hub.Module(module)
            embeddings = embed(sentences)
            session = tf.train.MonitoredSession()
        return lambda x: session.run(embeddings, {sentences: x})
    embed_fn = embed_useT(module_url)

    # Generate embeddings
    for company in tqdm(company_to_tweets.keys()):
        for date in company_to_tweets[company].keys():
            messages = []
            for j in range(len(company_to_tweets[company][date])):
                messages.append(' '.join(company_to_tweets[company][date][j]['text']))
                message_embeddings = embed_fn(messages)
            for k in range(len(company_to_tweets[company][date])):
                company_to_tweets[company][date][k]['embedding'] = list(message_embeddings[k])
    
    # Create date mapping
    date_universe = set()
    for company in company_to_price_df.keys():
        date_universe = date_universe.union(set(company_to_price_df[company].date))
    for company in company_to_tweets.keys():
        date_universe = date_universe.union(set(company_to_tweets[company].keys()))
    date_universe = sorted(list(date_universe))
    index_to_date = {i-5:d for i,d in enumerate(date_universe)}
    date_to_index = {d:i-5 for i,d in enumerate(date_universe)}

    # Calculate dimensions for tensor
    n_stocks = len(company_to_tweets.keys())
    n_days = len(date_universe)
    max_tweets = 0
    for c,d in itertools.product(company_to_tweets.keys(), date_universe):
        if d in company_to_tweets[c]:
            max_tweets = max(max_tweets, len(company_to_tweets[c][d]))
    # Create index mapping for stocks alphabetically
    company_to_index = {c:i for i,c in enumerate(sorted(list(company_to_tweets.keys())))}

    return company_to_price_df, company_to_tweets, date_universe, n_days, n_stocks, max_tweets

In [11]:
train_company_to_price_df, train_company_to_tweets, train_date_universe, train_n_days, train_n_stocks, train_max_tweets = prep_dataset(stocknet_dataset_filepath, train_start_date, train_end_date)
val_company_to_price_df, val_company_to_tweets, val_date_universe, val_n_days, val_n_stocks, val_max_tweets = prep_dataset(stocknet_dataset_filepath, val_start_date, val_end_date)
test_company_to_price_df, test_company_to_tweets, test_date_universe, test_n_days, test_n_stocks, test_max_tweets = prep_dataset(stocknet_dataset_filepath, test_start_date, test_end_date)

#Dataset and DataLoader

In [61]:
class StockDataset(Dataset):
    """Price dataset"""

    def __init__(self, company_to_price_df, company_to_tweets, date_universe, n_days, n_stocks, max_tweets):
        # Initialize class members
        self.n_stocks = n_stocks
        self.n_days = n_days
        self.max_tweets = max_tweets

        # Build maps
        self.company_to_index = {c:i for i,c in enumerate(sorted(list(company_to_tweets.keys())))}
        self.date_to_index = {d:i for i,d in enumerate(date_universe)}
        self.index_to_date = {i:d for i,d in enumerate(date_universe)}

        # Store data
        self.company_to_price_df = company_to_price_df
        self.company_to_tweets = company_to_tweets

    def __len__(self):
        return self.n_days-7

    def __getitem__(self, idx):
        """
        gets a price tensor of shape (n_stocks, 6, 3)
        gets a smi tensor of shape (n_stocks, 6, K, 512)
        """
        # Size of sliding window
        window = 6

        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Dates that we need to look up
        dates_range = [self.index_to_date[i] for i in range(idx + 1, idx + window + 1)]

        # Day after (for label)
        day_after = self.index_to_date[idx + window + 1]

        # Current day
        current_day = self.index_to_date[idx + window]

        # Which stocks are usable for these dates
        usable_stocks = torch.ones(self.n_stocks)

        # Labels
        labels = torch.zeros(self.n_stocks)

        # Get labels
        for company in self.company_to_price_df.keys():
            df = self.company_to_price_df[company]

            # Grab row with particular date
            post_row = df.loc[df['date'] == day_after]
            row = df.loc[df['date'] == current_day]
            c_index = self.company_to_index[company]

            if (len(post_row['adjust_close']) > 0) and (len(row['adjust_close']) > 0):
                close = np.zeros((1))
                close[0] = post_row['adjust_close']
                close[0] /= row['adjust_close']
                if close >= 1.0055:
                    labels[c_index] = 1
                elif close <= 0.995:
                    labels[c_index] = 0
                else:
                    usable_stocks[c_index] = 0
            else:
                usable_stocks[c_index] = 0

        # Get price data tensor: n_stocks, window, 3
        price_data = np.zeros((self.n_stocks, window, 3))
        for company in self.company_to_price_df.keys():
            df = self.company_to_price_df[company]

            prev_day = self.index_to_date[idx]
            prev_row = df.loc[df['date'] == prev_day]
            # Look up specific rows in DF
            for date_idx, date in enumerate(dates_range):

                # Grab row with particular date
                row = df.loc[df['date'] == date]
                c_index = self.company_to_index[company]

                if (len(row['high']) > 0) and (len(row['low']) > 0) and (len(row['adjust_close']) > 0) and (len(prev_row['adjust_close']) > 0):
                    price_data[c_index, date_idx, 0] = row['high']
                    price_data[c_index, date_idx, 1] = row['low']
                    price_data[c_index, date_idx, 2] = row['adjust_close']

                    price_data[c_index, date_idx, 0] /= prev_row['adjust_close']
                    price_data[c_index, date_idx, 1] /= prev_row['adjust_close']
                    price_data[c_index, date_idx, 2] /= prev_row['adjust_close']
                else:
                    usable_stocks[c_index] = 0
                
                prev_row = row

        # Extract tweets for specific window
        smi_data = np.zeros((self.n_stocks, window, self.max_tweets, 512))
        tweet_counts = np.zeros((self.n_stocks, window))
        for company in self.company_to_tweets.keys():

            # Look up tweets from specific days
            for date_idx, date in enumerate(dates_range):
                n_tweets = 0
                tweets = []
                c_index = self.company_to_index[company]
                if date in self.company_to_tweets[company]:
                    n_tweets = len(self.company_to_tweets[company][date])
                    tweets = [self.company_to_tweets[company][date][k]['embedding'] for k in range(n_tweets)]
                else:
                    usable_stocks[c_index] = 0
                tweet_counts[c_index, date_idx] = n_tweets
                if n_tweets == 0:
                    usable_stocks[c_index] = 0
                for i,embedding in enumerate(tweets): 
                    #stocks, day, lags, tweet, embedding
                    smi_data[c_index, date_idx, i, :] = embedding[:]

        usable_stocks = (usable_stocks == 1)

        m_mask = torch.zeros(6, self.n_stocks, self.max_tweets, 1)
        for t in range(6):
            for i in range(self.n_stocks):
                for k in range(self.max_tweets):
                    if k <= tweet_counts[i][t]:
                        m_mask[t][i][k][0] = 1

        #price_output = price_data[usable_stocks,:,:]
        #smi_output = smi_data[usable_stocks,:,:,:]
        #tweet_count = tweet_counts[usable_stocks,:]
        price_output = torch.tensor(price_data[usable_stocks,:,:])
        smi_output = torch.tensor(smi_data[usable_stocks,:,:,:])
        tweet_count = torch.tensor(tweet_counts[usable_stocks,:])
        m_mask = m_mask[:,usable_stocks,:,:]
        labels = labels[usable_stocks]
        
        # construct output
        return price_output, smi_output, tweet_count, usable_stocks, labels, m_mask

In [62]:
train_dataset = StockDataset(train_company_to_price_df, train_company_to_tweets, train_date_universe, train_n_days, train_n_stocks, train_max_tweets)
val_dataset = StockDataset(val_company_to_price_df, val_company_to_tweets, val_date_universe, val_n_days, val_n_stocks, val_max_tweets)
test_dataset = StockDataset(test_company_to_price_df, test_company_to_tweets, test_date_universe, test_n_days, test_n_stocks, test_max_tweets)

In [63]:
train_dataloader = DataLoader(train_dataset, batch_size=1,
                        shuffle=True, num_workers=0)

val_dataloader = DataLoader(val_dataset, batch_size=1,
                        shuffle=False, num_workers=0)

test_dataloader = DataLoader(test_dataset, batch_size=1,
                        shuffle=False, num_workers=0)

#Separator

In [64]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


#Training

In [65]:
mansf = MANSF(T=6,
              gru_hidden_size=64,
              attn_inter_size=32,
              use_embed_size=512,
              blend_size=32,
              gat_1_inter_size=32,
              gat_2_inter_size=32,
              leakyrelu_slope=0.01,
              elu_alpha=1.0,
              U=8)

In [66]:
mansf = mansf.to(device)

In [75]:
optimizer = optim.Adam(mansf.parameters(), lr=5e-4)
loss_fn = nn.BCELoss(reduction='mean')

In [76]:
train_acc_list = []
val_acc_list = []

In [ ]:
for epoch in range(18):
    mansf.train()
    correct = 0.0
    total = 0.0
    running_loss = 0.0
    for price, smi, n_tweets, usable_stocks, labels, m_mask in tqdm(train_dataloader):
        price = price.type(torch.FloatTensor)
        smi = smi.type(torch.FloatTensor)
        price = price.float()
        smi = smi.float()

        price = price.to(device)
        smi = smi.to(device)
        n_tweets = n_tweets.to(device)
        usable_stocks = usable_stocks.to(device)
        labels = labels.to(device)
        m_mask = m_mask.to(device)

        price = price.view(price.shape[1], price.shape[2], price.shape[3])
        smi = smi.view(smi.shape[1], smi.shape[2], smi.shape[3], smi.shape[4])
        #print(smi)
        n_tweets = n_tweets.view(n_tweets.shape[1], n_tweets.shape[2])
        usable_stocks = usable_stocks.view(usable_stocks.shape[1])
        m_mask = m_mask.view(m_mask.shape[1], m_mask.shape[2], m_mask.shape[3], m_mask.shape[4])

        smi = smi.permute(1, 0, 2, 3)

        m = []
        for t in range(6):
            m.append(smi[t])

        neighborhoods = torch.eye(87, 87)
        neighborhoods = neighborhoods.to(device)
        neighborhoods = neighborhoods[usable_stocks, :]
        neighborhoods = neighborhoods[:, usable_stocks]

        if price.shape[0] != 0:
            y = mansf(price, smi, m_mask, neighborhoods)
            loss = loss_fn(y.view(-1), labels.view(-1))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            correct += torch.sum(((y > 0.5).view(-1) == labels.view(-1))).item()
            total += len(y)
            running_loss = loss.item() * len(y)

    train_acc = correct / total
    train_acc_list.append(train_acc)

    mansf.eval()
    correct = 0.0
    total = 0.0
    for price, smi, n_tweets, usable_stocks, labels, m_mask in tqdm(val_dataloader):
        price = price.type(torch.FloatTensor)
        smi = smi.type(torch.FloatTensor)
        price = price.float()
        smi = smi.float()

        price = price.to(device)
        smi = smi.to(device)
        n_tweets = n_tweets.to(device)
        usable_stocks = usable_stocks.to(device)
        labels = labels.to(device)
        m_mask = m_mask.to(device)

        price = price.view(price.shape[1], price.shape[2], price.shape[3])
        smi = smi.view(smi.shape[1], smi.shape[2], smi.shape[3], smi.shape[4])
        n_tweets = n_tweets.view(n_tweets.shape[1], n_tweets.shape[2])
        usable_stocks = usable_stocks.view(usable_stocks.shape[1])
        m_mask = m_mask.view(m_mask.shape[1], m_mask.shape[2], m_mask.shape[3], m_mask.shape[4])

        smi = smi.permute(1, 0, 2, 3)

        m = []
        for t in range(6):
            m.append(smi[t])

        neighborhoods = torch.eye(87, 87)
        neighborhoods = neighborhoods.to(device)
        neighborhoods = neighborhoods[usable_stocks, :]
        neighborhoods = neighborhoods[:, usable_stocks]

        if price.shape[0] != 0:
            y = mansf(price, smi, m_mask, neighborhoods)
            #print(y)
            correct += torch.sum((y > 0.5).view(-1) == labels.view(-1)).item()
            total += len(y)

    val_acc = correct / total
    val_acc_list.append(val_acc)

    print('epoch:', epoch, 'loss:', running_loss, 'train_acc:', train_acc, 'val_acc:', val_acc)

tensor([[0.5417],
        [0.5417],
        [0.5416],
        [0.5419],
        [0.5416],
        [0.5416],
        [0.5418],
        [0.5418],
        [0.5417],
        [0.5418],
        [0.5417],
        [0.5416],
        [0.5419]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.5416],
        [0.5417],
        [0.5416],
        [0.5416],
        [0.5419],
        [0.5416],
        [0.5418],
        [0.5419]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.5417],
        [0.5419],
        [0.5416],
        [0.5418],
        [0.5418],
        [0.5419]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.5417],
        [0.5416],
        [0.5416],
        [0.5419],
        [0.5419],
        [0.5417],
        [0.5416],
        [0.5416],
        [0.5418],
        [0.5418]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor([[0.5417],
        [0.5416],
        [0.5417],
        [0.5416],
        [0.5418],
        [0.5416],
        [0.5418],
        [0.5418],
        [0

In [ ]:
mansf.eval()

price, smi, n_tweets, usable_stocks, labels, m_mask = next(iter(val_dataloader))

price = price.type(torch.FloatTensor)
smi = smi.type(torch.FloatTensor)

price = price.to(device)
smi = smi.to(device)
n_tweets = n_tweets.to(device)
usable_stocks = usable_stocks.to(device)
labels = labels.to(device)
m_mask = m_mask.to(device)

price = price.view(price.shape[1], price.shape[2], price.shape[3])
smi = smi.view(smi.shape[1], smi.shape[2], smi.shape[3], smi.shape[4])
n_tweets = n_tweets.view(n_tweets.shape[1], n_tweets.shape[2])
usable_stocks = usable_stocks.view(usable_stocks.shape[1])
m_mask = m_mask.view(m_mask.shape[1], m_mask.shape[2], m_mask.shape[3], m_mask.shape[4])

smi = smi.permute(1, 0, 2, 3)

m = []
for t in range(6):
    m.append(smi[t])

neighborhoods = torch.eye(87, 87)
neighborhoods = neighborhoods.to(device)
neighborhoods = neighborhoods[usable_stocks, :]
neighborhoods = neighborhoods[:, usable_stocks]

y = mansf(price, smi, m_mask, neighborhoods)

In [ ]:
print(y)

In [ ]:
print(labels)

#Figures

In [ ]:
def plot(X, Y, xlabel, ylabel, legend, title):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    for i in range(len(Y)):
        ax.plot(X, Y[i], label=legend[i])

    plt.grid(color='0.95')
    plt.legend()
    ax.set(xlabel=xlabel, ylabel=ylabel, title=title)

In [ ]:
"""plot(range(18),
     [train_acc_list, val_acc_list],
     'epoch',
     'accuracy',
     ['training accuracy', 'validation accuracy'],
     'accuracy vs. epoch')"""